# List Nodes in the Production Environment

Connect to the DataONE production environment, retrieve the list of nodes, and print out the response document.

Uses the [`CNCore.listNodes()`](https://dataone-architecture-documentation.readthedocs.io/en/latest/apis/CN_APIs.html#CNCore.listNodes) API.

In [ ]:
# notebook utils contains some convenience methods, see notebook_utils/__init__.py
import notebook_utils as nbu

from d1_client import cnclient_2_0

cn_base_url = nbu.ENVIRONMENTS["production"]
print("Using base url of: {0}\n".format(cn_base_url))

cncli = cnclient_2_0.CoordinatingNodeClient_2_0(base_url=cn_base_url)
nodes = cncli.listNodes()

print("XML Response:")
print(nbu.asXml(nodes, max_lines=40))

## Formatted list of Nodes

Print out a formatted list of Member Nodes in the environment

In [ ]:
# Print a formatted list of nodes, showing only MNs:
i = 0
for node in nodes.node:
    if node.type.upper() == "MN":
        i += 1
        print(f"{i:02d}: {nbu.propertyStr(node.identifier):25}{node.baseURL:<55}{node.state:<5}{node.name}")


## Mapping the nodes

Approximate locations of the nodes are available in the node document. Making a map of the nodes is straight forward:

In [ ]:
from ipyleaflet import Map, Marker, MarkerCluster

def getCNProperty(node, name):
    #Note: 'property' is a reserved name in Pyxb, so it is renamed to "property_"
    for p in node.property_:
        if p.key == name:
            return p.value()
    return None

m = Map(center=(0, -50), zoom=1)
m.layout.width = "100%"
m.layout.height = "600px"

for node in nodes.node:
    if node.type.upper() == "MN":
        coord = getCNProperty(node, "CN_location_lonlat")
        if coord is not None:
            lon,lat = coord.split(",")
            marker = Marker(location=(lat, lon), title=node.identifier.value(), draggable=False)
            m.add_layer(marker)
m